In [ ]:
# !pip install emoji

import pandas as pd
import re
import emoji
from binance.client import Client
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [348]:
data = pd.read_csv('G-D.csv',engine='python', quotechar='"', encoding='utf-8', on_bad_lines='skip')

data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680.0,2643.0,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770.0,1145.0,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,576.0,1.0,0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26 10:15:26,236.0,1829.0,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731.0,46.0,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [349]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174397 entries, 0 to 174396
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_name         174336 non-null  object 
 1   user_location     85028 non-null   object 
 2   user_description  159563 non-null  object 
 3   user_created      170627 non-null  object 
 4   user_followers    169820 non-null  float64
 5   user_friends      169820 non-null  float64
 6   user_favourites   169820 non-null  object 
 7   user_verified     169820 non-null  object 
 8   date              169820 non-null  object 
 9   text              169820 non-null  object 
 10  hashtags          169013 non-null  object 
 11  source            169013 non-null  object 
 12  is_retweet        168954 non-null  object 
dtypes: float64(2), object(11)
memory usage: 17.3+ MB


In [350]:
data.isnull().sum()

user_name              61
user_location       89369
user_description    14834
user_created         3770
user_followers       4577
user_friends         4577
user_favourites      4577
user_verified        4577
date                 4577
text                 4577
hashtags             5384
source               5384
is_retweet           5443
dtype: int64

In [351]:
data.describe()

,user_followers,user_friends
count,1.698200e+05,169820.000000
mean,1.059798e+04,771.399694
std,1.308698e+05,2677.266627
min,0.000000e+00,0.000000
25%,1.190000e+02,9.000000
50%,5.460000e+02,122.000000
75%,1.956000e+03,606.000000
max,1.878937e+07,254276.000000


In [352]:
data.describe(include='object')

,user_name,user_location,user_description,user_created,user_favourites,user_verified,date,text,hashtags,source,is_retweet
count,174336,85028,159563,170627,169820,169820,169820,169820,169013,169013,168954
unique,39502,10595,33318,37110,17478,61,142505,167092,40810,720,1
top,Ethereum Yoda,USA,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,0,False,2023-03-01 14:00:03,Top 5 #cryptocurrency #price jumps in last min...,['Bitcoin'],Twitter Web App,False
freq,8721,2146,8714,8714,24354,168353,14,287,29312,53653,168954


In [353]:
data['user_created'] = pd.to_datetime(data['user_created'], errors='coerce')
data['date'] = pd.to_datetime(data['date'], errors='coerce')

data['user_verified'] = data['user_verified'].astype(bool)
data['is_retweet'] = data['is_retweet'].astype(bool)

data['user_favourites'] = pd.to_numeric(data['user_favourites'], errors='coerce')

data = data.dropna(subset=['text', 'date'])
data = data.drop(columns=['user_location'])

In [354]:
def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace='')
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"[^\w\s\-]", "", text)  
    text = re.sub(r"\s+", " ", text).strip()
    return text

text_columns = ['user_name', 'text', 'user_description', 'source']

for col in text_columns:
    data[col] = data[col].apply(lambda x: clean_text(x) if pd.notna(x) else None)
    data[col] = data[col].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)
    most_common = data[col].mode()[0]
    data[col] = data[col].fillna(most_common)

In [367]:
most_common_description = data["hashtags"].mode()[0]
data["hashtags"] = data["hashtags"].fillna(most_common_description)

In [356]:
data.head()

,user_name,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,chefsam,culinarian hot sauce artisan kombucha brewer n...,2011-03-23 03:50:13,4680.0,2643.0,6232.0,True,2023-03-01 23:59:59,which books should i think about reading next,['bitcoin'],twitter for iphone,False
1,roy,truth-seeking pleb science nature stacker free...,2022-01-30 17:41:41,770.0,1145.0,9166.0,True,2023-03-01 23:59:47,i appreciate the message but not a fan of the ...,['Bitcoin'],twitter for iphone,False
2,ethereum yoda,up or down price matters not,2022-07-24 04:50:18,576.0,1.0,0.0,True,2023-03-01 23:59:42,price update 166402 usd 0070428 follow for rec...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",twitter web app,False
3,viction,up or down price matters not,2010-03-26 10:15:26,236.0,1829.0,2195.0,True,2023-03-01 23:59:36,coindashboard v30 is here available on ios and...,['Bitcoin'],twitter for android,False
4,rosie,the flower language of jasmine is loyalty resp...,2013-02-16 09:57:56,12731.0,46.0,134.0,True,2023-03-01 23:59:32,short term fractal 4h in lower timeframe price...,"['Bitcoin', 'BTC']",twitter web app,False


In [357]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169761 entries, 0 to 174396
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_name         169761 non-null  object        
 1   user_description  169761 non-null  object        
 2   user_created      169761 non-null  datetime64[ns]
 3   user_followers    169761 non-null  float64       
 4   user_friends      169761 non-null  float64       
 5   user_favourites   169761 non-null  float64       
 6   user_verified     169761 non-null  bool          
 7   date              169761 non-null  datetime64[ns]
 8   text              169761 non-null  object        
 9   hashtags          169761 non-null  object        
 10  source            169761 non-null  object        
 11  is_retweet        169761 non-null  bool          
dtypes: bool(2), datetime64[ns](2), float64(3), object(5)
memory usage: 14.6+ MB


In [358]:
print(data.isnull().sum())
print(data.shape)

user_name           0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64
(169761, 12)


In [359]:
data.describe()

,user_created,user_followers,user_friends,user_favourites,date
count,169761,1.697610e+05,169761.000000,1.697610e+05,169761
mean,2019-07-29 22:10:27.212380672,1.060164e+04,771.526534,6.246872e+03,2023-03-01 19:17:30.301453312
min,2006-09-04 17:48:16,0.000000e+00,0.000000,0.000000e+00,2023-02-25 20:49:09
25%,2017-12-08 01:59:39,1.190000e+02,9.000000,1.100000e+01,2023-02-28 04:22:13
50%,2021-05-26 09:12:58,5.460000e+02,122.000000,2.810000e+02,2023-03-01 16:44:28
75%,2022-07-11 05:24:30,1.956000e+03,606.000000,3.304000e+03,2023-03-03 12:19:28
max,2023-03-05 23:20:37,1.878937e+07,254276.000000,1.083014e+06,2023-03-05 23:59:56
std,NaN,1.308924e+05,2677.582724,2.183784e+04,NaN


In [360]:
data.describe(include='object')

,user_name,user_description,text,hashtags,source
count,169761,169761,169761,169761,169761
unique,35053,31433,140802,40809,716
top,ethereum yoda,up or down price matters not,top 5 jumps in last minute on,['Bitcoin'],twitter web app
freq,9794,28943,396,30119,54475


In [ ]:
X_num = data[['user_followers', 'user_friends', 'user_favourites']]
X_num = X_num.fillna(0)

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

In [370]:
data.describe()

,user_created,user_followers,user_friends,user_favourites,date
count,169761,1.697610e+05,169761.000000,1.697610e+05,169761
mean,2019-07-29 22:10:27.212380672,1.060164e+04,771.526534,6.246872e+03,2023-03-01 19:17:30.301453312
min,2006-09-04 17:48:16,0.000000e+00,0.000000,0.000000e+00,2023-02-25 20:49:09
25%,2017-12-08 01:59:39,1.190000e+02,9.000000,1.100000e+01,2023-02-28 04:22:13
50%,2021-05-26 09:12:58,5.460000e+02,122.000000,2.810000e+02,2023-03-01 16:44:28
75%,2022-07-11 05:24:30,1.956000e+03,606.000000,3.304000e+03,2023-03-03 12:19:28
max,2023-03-05 23:20:37,1.878937e+07,254276.000000,1.083014e+06,2023-03-05 23:59:56
std,NaN,1.308924e+05,2677.582724,2.183784e+04,NaN


In [371]:
data.describe(include='object')

,user_name,user_description,text,hashtags,source
count,169761,169761,169761,169761,169761
unique,35053,31433,140802,40809,716
top,ethereum yoda,up or down price matters not,top 5 jumps in last minute on,['Bitcoin'],twitter web app
freq,9794,28943,396,30119,54475


In [ ]:
api_key = "TLXuzjmtV05vOkRYBaeQXWg3p3A6FbsEEVur2ixrJB7bBjY76G0IS1sp4ALhKQjq"
api_secret = "7DNgiq61ajYq7Ul6l8wW5sfUqgSr742PHHqMZgh3Gcn0hnrssvMqKWF80XhKsUtT"

client = Client(api_key, api_secret)

symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
start_time = "2023-02-25 00:00:00"
end_time = "2023-03-05 23:59:59"

klines = client.get_historical_klines(symbol, interval, start_str=start_time, end_str=end_time)

df_price = pd.DataFrame(klines, columns=[
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
])

df_price["date"] = pd.to_datetime(df_price["open_time"], unit="ms")
df_price["price"] = df_price["close"].astype(float)
df_price = df_price[["date", "price"]]


In [ ]:
tweets = data.copy()
tweets['date_minute'] = tweets['date'].dt.floor('min')
tweets_prices = pd.merge(tweets, df_price, how='left', left_on='date_minute', right_on='date')
tweets_prices.drop(columns=['date_minute', 'date_y'], inplace=True)
tweets_prices.rename(columns={'date_x': 'date'}, inplace=True)
tweets_prices[['date', 'text', 'price']].head()

,date,text,price
0,2023-03-01 23:59:59,which books should i think about reading next,23628.97
1,2023-03-01 23:59:47,i appreciate the message but not a fan of the ...,23628.97
2,2023-03-01 23:59:42,price update 166402 usd 0070428 follow for rec...,23628.97
3,2023-03-01 23:59:36,coindashboard v30 is here available on ios and...,23628.97
4,2023-03-01 23:59:32,short term fractal 4h in lower timeframe price...,23628.97


In [363]:
tweets_prices.isnull().sum()

user_name           0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
price               0
dtype: int64

In [364]:
tweets_prices.to_csv("tweets_with_btc_price.csv", index=False)